In [2]:
!pip install datasets deepeval

  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of opentelemetry-exporter-otlp-proto-grpc to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.9/508.9 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 11.6 MB/s

In [2]:
import os
import argparse

import numpy as np
from datasets import load_dataset
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCase
from deepeval.test_case import LLMTestCaseParams
from dotenv import load_dotenv
from openai import OpenAI
from scipy.stats import ttest_rel, ttest_1samp

load_dotenv()
client = OpenAI()

style_matching_metric_geval = GEval(
    name="Style Matching",
    criteria="Определи, насколько последняя фраза (actual output) делает анекдот смешным, логичным и остроумным:",

    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT, LLMTestCaseParams.EXPECTED_OUTPUT]
)

naturalness_metric_geval = GEval(
    name="Naturalness",
    criteria="Оцени, насколько последняя фраза (actual output) звучит естественно в контексте предшествующего диалога:",

    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT, LLMTestCaseParams.EXPECTED_OUTPUT]
)

system_message1 = {"role": "system",
                   "content": "Ты чат бот, который генерирует анекдоты. "
                              "Продолжи диалог одной репликой так, чтоб получился анекдот. "
                              "Example input: - Я не могу выйти на работу, потому что жена сломала два ребра. - А при чём тут ты? "
                              "Expected output: - Мои!"}
system_message2 = {"role": "system",
                   "content": "Ты чат бот, который генерирует диалоги. Продолжи диалог"}


def prepare_openai_jokes_dataset(num_samples):
    ds = load_dataset("inkoziev/jokes_dialogues")
    ds_df = ds['train'].to_pandas()

    # print(ds_df)
    unique_chat_ids = ds_df['src_hash'].unique()
    # unique_chat_ids = set(ds['train']['src_hash'])
    sampled_chat_ids = np.random.choice(list(unique_chat_ids), num_samples, replace=False)

    openai_chat_examples = []
    for chat_id in sampled_chat_ids:
        reply_nums = ds_df[ds_df['src_hash'] == chat_id]['reply_num'].values
        reply_nums = np.sort(reply_nums)
        messages = [system_message1]
        if reply_nums[0] != 1:
            continue
        message1 = ds_df[(ds_df['src_hash'] == chat_id) & (ds_df['reply_num'] == 1)]['context'].values[0]
        messages.append({"role": "assistant" if 0 % 2 == reply_nums[-1] % 2 else "user",
                         "content": message1})

        for reply_num in reply_nums:
            message = ds_df[(ds_df['src_hash'] == chat_id) & (ds_df['reply_num'] == reply_num)]['utterance'].values[0]
            message_metadata = {"role": "assistant" if reply_num % 2 == reply_nums[-1] % 2 else "user",
                                "content": message}
            if message_metadata['role'] == 'assistant':
                if reply_num == reply_nums[-1]:
                    message_metadata['weight'] = 1
                else:
                    message_metadata['weight'] = 0
            messages.append(message_metadata)

        openai_chat_example = {"messages": messages}
        openai_chat_examples.append(openai_chat_example)
    return openai_chat_examples


def get_model_predictions(gt_examples, system_message):
    predicted_examples = []
    for i_example in range(len(gt_examples)):
        completion = client.chat.completions.create(
            model="ft:gpt-4o-mini-2024-07-18:personal::AgreyVd1",
            messages=[system_message] + gt_examples[i_example]['messages'][1:-1],
        )
        # predicted_examples.append(completion.choices[0].message.content)
        predicted_example = [system_message] + gt_examples[i_example]['messages'][1:-1]
        predicted_example.append({"role": "assistant", "content": completion.choices[0].message.content})
        predicted_example = {"messages": predicted_example}
        predicted_examples.append(predicted_example)

    return predicted_examples


def evaluate_metrics(gt_examples, predicted_examples1, predicted_examples2):
    scores1 = []
    scores2 = []

    naturalness_scores1 = []
    naturalness_scores2 = []

    for i_example in range(len(gt_examples)):
        test_case1 = LLMTestCase(
            input="\n".join([msg['content'] for msg in predicted_examples1[i_example]['messages'][1:-1]]),
            actual_output=predicted_examples1[i_example]['messages'][-1]['content'],
            expected_output=gt_examples[i_example]['messages'][-1]['content'],
        )
        test_case2 = LLMTestCase(
            input="\n".join([msg['content'] for msg in predicted_examples2[i_example]['messages'][1:-1]]),
            actual_output=predicted_examples2[i_example]['messages'][-1]['content'],
            expected_output=gt_examples[i_example]['messages'][-1]['content'],
        )

        style_matching_metric_geval.measure(test_case1)
        predicted_examples1[i_example]['fun_score'] = style_matching_metric_geval.score
        scores1.append(style_matching_metric_geval.score)

        style_matching_metric_geval.measure(test_case2)
        predicted_examples2[i_example]['fun_score'] = style_matching_metric_geval.score
        scores2.append(style_matching_metric_geval.score)


        naturalness_metric_geval.measure(test_case1)
        naturalness_scores1.append(naturalness_metric_geval.score)

        naturalness_metric_geval.measure(test_case2)
        naturalness_scores2.append(naturalness_metric_geval.score)

    return gt_examples, predicted_examples1, predicted_examples2, scores1, scores2, naturalness_scores1, naturalness_scores2


def main(args):
    gt_examples = prepare_openai_jokes_dataset(args)
    print("============== GT EXAMPLES ===============")
    for i_example in range(len(gt_examples)):
        print(gt_examples[i_example])
        print("-----------")
    predicted_examples1 = get_model_predictions(gt_examples, system_message1)
    predicted_examples2 = get_model_predictions(gt_examples, system_message2)
    print("=============== GT and Predicted EXAMPLES ======================")
    for i_example in range(len(gt_examples)):
        print(gt_examples[i_example])
        print(predicted_examples1[i_example])
        print(predicted_examples2[i_example])
        print("-----------")
    gt_examples, predicted_examples1, predicted_examples2, scores1, scores2, naturalness_scores1, naturalness_scores2 = evaluate_metrics(gt_examples, predicted_examples1, predicted_examples2)

    print("=============== GT and Predicted EXAMPLES with METRICS ======================")
    for i_example in range(len(gt_examples)):
        print(gt_examples[i_example])
        print(predicted_examples1[i_example])
        print(predicted_examples2[i_example])
        print("-----------")

    print("=============== Scores ======================")
    ttest_res1 = ttest_1samp(scores1, 0.5)
    print("SCORE1:", np.mean(scores1), scores1)
    print("TTEST1:", ttest_res1)
    ttest_res2 = ttest_1samp(scores2, 0.5)
    print("SCORE2:", np.mean(scores2), scores2)
    print("TTEST2:", ttest_res2)

    rel_ttest_res = ttest_rel(scores1, scores2)

    print("=============== Rel Scores ======================")
    print("TTEST:", rel_ttest_res)

    print("=============== Naturalness Scores ======================")
    print("Naturalness SCORE1:", np.mean(naturalness_scores1), naturalness_scores1)
    print("Naturalness TTEST1:", ttest_1samp(naturalness_scores1, 0.5))
    print("Naturalness SCORE2:", np.mean(naturalness_scores2), naturalness_scores2)
    print("Naturalness TTEST2:", ttest_1samp(naturalness_scores2, 0.5))

    rel_ttest_naturalness = ttest_rel(naturalness_scores1, naturalness_scores2)
    print("Naturalness Rel TTEST:", rel_ttest_naturalness)

if __name__ == '__main__':
    num_samples = 30
    main(num_samples)


============== GT EXAMPLES ===============
{'messages': [{'role': 'system', 'content': 'Ты чат бот, который генерирует анекдоты. Продолжи диалог одной репликой так, чтоб получился анекдот. Example input: - Я не могу выйти на работу, потому что жена сломала два ребра. - А при чём тут ты? Expected output: - Мои!'}, {'role': 'user', 'content': '- Как дела?'}, {'role': 'assistant', 'content': 'На ту же букву, что и хорошо.', 'weight': 1}]}
-----------
{'messages': [{'role': 'system', 'content': 'Ты чат бот, который генерирует анекдоты. Продолжи диалог одной репликой так, чтоб получился анекдот. Example input: - Я не могу выйти на работу, потому что жена сломала два ребра. - А при чём тут ты? Expected output: - Мои!'}, {'role': 'user', 'content': '- Алексей Петрович, вы допускаете телессные наказания в воспитательном процессе?'}, {'role': 'assistant', 'content': 'Если речь идет о моем Вовочке, то я его луплю только в целях самозащиты.', 'weight': 1}]}
-----------
{'messages': [{'role': 'sys

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

=============== GT and Predicted EXAMPLES ======================
{'messages': [{'role': 'system', 'content': 'Ты чат бот, который генерирует анекдоты. Продолжи диалог одной репликой так, чтоб получился анекдот. Example input: - Я не могу выйти на работу, потому что жена сломала два ребра. - А при чём тут ты? Expected output: - Мои!'}, {'role': 'user', 'content': '- Как дела?'}, {'role': 'assistant', 'content': 'На ту же букву, что и хорошо.', 'weight': 1}]}
{'messages': [{'role': 'system', 'content': 'Ты чат бот, который генерирует анекдоты. Продолжи диалог одной репликой так, чтоб получился анекдот. Example input: - Я не могу выйти на работу, потому что жена сломала два ребра. - А при чём тут ты? Expected output: - Мои!'}, {'role': 'user', 'content': '- Как дела?'}, {'role': 'assistant', 'content': 'На работе уволили. Жена говорит, теперь надо найти новую.'}]}
{'messages': [{'role': 'system', 'content': 'Ты чат бот, который генерирует диалоги. Продолжи диалог'}, {'role': 'user', 'cont

Output()

...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

=============== GT and Predicted EXAMPLES with METRICS ======================
{'messages': [{'role': 'system', 'content': 'Ты чат бот, который генерирует анекдоты. Продолжи диалог одной репликой так, чтоб получился анекдот. Example input: - Я не могу выйти на работу, потому что жена сломала два ребра. - А при чём тут ты? Expected output: - Мои!'}, {'role': 'user', 'content': '- Как дела?'}, {'role': 'assistant', 'content': 'На ту же букву, что и хорошо.', 'weight': 1}]}
{'messages': [{'role': 'system', 'content': 'Ты чат бот, который генерирует анекдоты. Продолжи диалог одной репликой так, чтоб получился анекдот. Example input: - Я не могу выйти на работу, потому что жена сломала два ребра. - А при чём тут ты? Expected output: - Мои!'}, {'role': 'user', 'content': '- Как дела?'}, {'role': 'assistant', 'content': 'На работе уволили. Жена говорит, теперь надо найти новую.'}], 'fun_score': 0.38996707134886777}
{'messages': [{'role': 'system', 'content': 'Ты чат бот, который генерирует диа

Стиль: Первая модель (SCORE1: 0.5743) показывает лучшее соответствие стилю анекдотов по сравнению со второй моделью (SCORE2: 0.2851). Разница значительная.

Естественность: Оценки естественности двух моделей близки (SCORE1: 0.4232, SCORE2: 0.4438), различия несущественны.

Итого первая модель лучше справляется с задачей генерации анекдотов в заданном стиле, тогда как естественность завершений у обеих моделей схожа.

Стиль: Оценки двух моделей практически идентичны (SCORE1: 0.3473, SCORE2: 0.3467). Различия между ними несущественны.

Естественность: Вторая модель показывает немного более высокую естественность (SCORE2: 0.3391) по сравнению с первой (SCORE1: 0.2965), но разница незначительна.

Итого, обе модели демонстрируют схожие результаты в передаче стиля, а вторая модель незначительно превосходит первую по естественности. В целом результаты хуже базового решения